In [ ]:
import requests
import pandas as pd
from prefect import flow, task, get_run_logger

In [ ]:
base_url = "http://air4thai.pcd.go.th"
path = "/services/getNewAQI_JSON.php"

In [ ]:
#logger = get_run_logger()
response = requests.get(url=base_url+path)
response.raise_for_status()
AQI_stats = response.json()
data = AQI_stats['stations']
print(data)
#logger.info(AQI_stats)

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
df.info()

In [ ]:
expanded_aqi = pd.json_normalize(df['AQILast'])
expanded_aqi

In [ ]:
df = pd.concat([df, expanded_aqi], axis=1)
df

In [ ]:
df['time'] = df['time'].mode()[0]
df['date'] = df['date'].mode()[0]
df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df[['year', 'month', 'day', 'hour']] = df['timestamp'].apply(
    lambda ts: pd.Series([ts.year, ts.month, ts.day, ts.hour])
)
df[['PM25.color_id', 'PM25.aqi']] = df[['PM25.color_id', 'PM25.aqi']].astype(int)

In [ ]:
df

In [ ]:
df[[
        'timestamp', 'year', 'month', 'day', 'hour',
        'stationID', 'nameTH', 'nameEN', 'areaTH', 'areaEN',
        'stationType', 'lat', 'long',
        'PM25.color_id', 'PM25.aqi'
]]